# A simple job shop example
In this notebook, we show you how to model and solve a simple job shop problem with PyJobShop.

A scheduling problem consists of three main elements:
- **Machine**: a machine is a unary resource that can process an operation.
- **Operation**: an operation is a processing task that needs to be completed.
- **Job**: this represents the collection of operations that need to be performed, as is used to measure performance such as completion times or tardiness.

In the job shop problem, there are $n$ jobs and $m$ machines. For each job, we have a sequence of operations
Let's consider the job shop example from [Google OR-Tools](https://developers.google.com/optimization/scheduling/job_shop).

In [45]:
data = [
    [(0, 3), (1, 2), (2, 2)],
    [(0, 2), (2, 1), (1, 4)],
    [(1, 4), (2, 3)],
]

NUM_MACHINES = 3

[autoreload of docplex.cp.expression failed: Traceback (most recent call last):
  File "/Users/leonlan/Dropbox/PyJobShop/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/leonlan/Dropbox/PyJobShop/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/leonlan/Dropbox/PyJobShop/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/leonlan/Dropbox/PyJobShop/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/leonlan/Dropbox/PyJobShop/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'CpoValue' object layout dif

Here we have three jobs. Each job consists of a list of tuples, each of which represents an operation, and the first index denotes the machine that can process it and the second 

In [46]:
from pyjobshop import Model, TimingPrecedence, plot, result2solution

In [47]:
model = Model()

In [48]:
machines = [model.add_machine() for _ in range(NUM_MACHINES)]

In [49]:
jobs = [model.add_job() for _ in range(len(data))]

In [50]:
for job_idx, operations in enumerate(data):
    ops = []

    for machine_idx, duration in operations:
        operation = model.add_operation()
        model.assign_job_operations(jobs[job_idx], [operation])
        model.add_processing_time(machines[machine_idx], operation, duration)

    for idx in range(len(ops)):
        model.add_timing_precedence(
            ops[idx],
            ops[idx + 1],
            TimingPrecedence.END_BEFORE_START,
        )

In [51]:
result = model.solve()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
data = model.data()
solution = result2solution(data, result)

plot(data, solution)